In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
BY_ACCOUNT = dataiku.Dataset("BY_ACCOUNT")
BY_ACCOUNT_df = BY_ACCOUNT.get_dataframe()

df = BY_ACCOUNT_df

In [5]:
customers = df.CUSTOMER.unique()
len(customers)

657099

In [8]:
import string
import nltk
from nltk.tag import pos_tag

df_dict = pd.DataFrame(nltk.corpus.words.words(), columns=['word'])
df_dict['word'] = df_dict['word'].str.upper()

In [14]:
class Common_Word:
    
    def __init__(self, word):
        
        self.word = word
        self.count = 1
        

class Common_Word_List:
        
    def __init__(self):
        
        self.values = []
        

    def add_word(self, word):

        found = False
        for w in self.values:
            if w.word==word:
                w.count+=1
                found = True

        if not found:
            self.values.append(Common_Word(word))
        

words = ['APPLE', 'PEAR', 'APPLE', 'ORANGE', 'PEAR']
        


for w in words:
    
    
    
for w in list_.values:
    print(w.word, w.count)

APPLE 2
PEAR 2
ORANGE 1


In [18]:
list_ = Common_Word_List()

for c in customers:
    
    c_str = c.translate(str.maketrans('', '', string.punctuation))
    f = c_str.split()
    
    for w in f:
        if (len(w)>1) and (not w.isnumeric()):
            list_.add_word(w)
    idx+=1
    
    if (idx % 1000 == 0):
        print(idx, len(list_.values))

    if idx>10000:
        break;
        
for w in list_.values:
    if w.count>20:
        print(w.word, w.count)

BERGE TRANSPORT
COUNTY OF MOBILE
IEC INC
BUYBACKBOY LLC
PJF ENTERPRISES INC
G  A ADVERTISING
BAUMGARDNER MOTORS INC
MULTITECH SEALERS INC
SINAK PLUMBING CO INC
TRICOUNTY ENGINEERING INC
THE ELLIOTT COMPANY
SERIOUS GRIP AND ELECTRIC
LT FOOD DISTRIBUTOR INC
WALL FX LTD
PERFORMANCE TESTING
HERITAGE HAVEN INC
INSANELY FAST HOME SOLUTIONS INC
S  K SANITATION INC
TK VISION HOLDINGS INCORPORATED
PAYNE R K REFRIGERATION  ELECTRICAL INC
MICRO INFORMATICA LLC
MARATHON FLORIST INC
SPINNING CRANE WORKS LLC
FALLS METAL FABRICATORS  INDUSTRIAL SERVICES LLC
MCKAY ASSOCIATES
UNTOUCHABLES TRANSPORTATION LLC
PWC MANAGEMENT LLC
KOLDAIR HEATING  AIR CONDITIONING INC
KOLDAIR HEATING  AIR CONDITIO
A 2 Z MECHANICAL CONTRACTORS INC
A 2 Z MECHANICAL CONTRACTORS
SOLID GROUND LANDSCAPE CONSTRUCTION INC
RUNWENATCHEE LLC
EXPLORE JLM IL INC
VAN SARK INC
THE READING CORNER
REDMOND REACH
BLANKENSHIP SERVICES LLC
ENDURANCE MACHINE INC
HANSON HEATING AIR CONDITIONING
TOWNEHOUSE MANAGEMENT ASSOCIATES
CLOCK CONTRACTING A

CHARLOTTESVILLE SANITARY SUPPLY CORP
LAR SOLUTIONS CO LLC
TOTAL LAND AND CONSTRUCTION LLC
ZEBRA LAWN CARE INC
PLAZA MAINTENANCE INC
COBBLESTONE MILL WOODWOR 30107
COBBLESTONE MILL WOODWORKS INC
AMERICAN DISPOSAL SERVICES INC
RAINBOW MOTORS INC
ROLLE WITH ME TRAVEL LLC
KRB REALTY LLC
PRISTINE SERENITY MED SPA LLC
RPS SERVICES LLC
A S STUCCO
OCD
WILMINGTON CHURCH OF CHRIST CHRISTIAN
ROLLS ON THE FLY INC
LAKESIDE MANAGEMENT
CITY OF SAVOY
MARIANETTI MOTORS INC
MEDICAL HOME CARE INC
MODINE MANUFACTURING COMPANY INC
AMERICAN RESTORATION  MAINTENANCE COR
GRANITE RUN PONTIACBUICKGMC INC
PETERSEN INC OF WORTHINGTON
NICKS DRYER VENT CLEANING INC
ANCHORAGE NEIGHBORHOOD HOUSING SERVICES INC
SAN GABRIEL VALLEY REGIONAL OCCUPATIONAL PROGRAM EDUCATION FOUNDATION
S  L ZEPPETELLI INC
T S I AMUSEMENTS COMPANY INC
SOUTHERN NEW HAMPSHIRE PLUMBING HEATING COOLING LLC
HEARTLAND INDUSTRIES INC
COMMUNITY ELECTRIC INC
BAILEY BROTHERS ENTERPRISES INC
MS INCHARGE ENT INC
PIONEER WELLS INC
ALPHA LEAK DETECTION SE

MHB ENTERPRISES LLC
AD MOHR TRUCKING LLC
ROCK  ROLL EXPRESS
TIKIGAQ CORPORATION
CTEK CONSTRUCTION INC
STROTHER TIMBER COMPANY LLC
NEXT LEVEL FINANCIAL  ADMINISTRATIVE SERVICES LLC
QUALITY GARAGE DOORS SERVICES LLC
MATT MILLER TRUCKING LLC
DISASTER MASTERS INC
DUBUQUE COUNTY HISTORICAL SOCIETY
JH WILLIAMS ENTERPRISES INC
DAN SOLID SURFACES INC
BIGHAM RANCH LLC
JC CONCRETE
TRUCKERS HOME OFFICE INC
VORTEX WRAP PRIVACY
MMAC
ONTRACK RAILWAY OPERATIONS LTD
HERITAGE STONE AND TILE INC
LOOK FAMILY EXTERIORS
JANEGO CONSTRUCTION INC
UNITED CHEVROLET BUICK GMC
OMEARA VOLKSWAGEN
COLORGRAPHIX
SHOFLO INC
NORTHERN VA CIGARETTE TAX BD
FULL CIRCLE HOME HEALTH CARE LLC
CONTRACT APPLIANCE SALES INC
MAK HOMES INC
JIMS EZ SHOP LLC
CUSTOMARY PROFESSIONALS
URBAN PROMISE MINISTRIES INC
NUMB3RS LLC
KURTEX
JOE WINDOWS CLEANING
PROFESSIONAL UTILITY SERVICES
PLAQUEMINES PARISH SCHOOLS
EAGLE RIVER POLARIS AND ARCTIC CAT LLC
AES OF THE MIDWEST INC
A E S OF MIDWEST INC
GIVE ME BEAUTY GLAM BAR LLC
W H JEFFERSON FUNER

CHARLTON CONSTRUCTION LLC
LC QUALITY INC
TOMMYS WELDING LIMITED
SIRI CONTRACTING LLC
SIRI CONTRACTING  LLC
C L WILLIAMS CONTRACTING
ORNNELL FIRE SPRINKLER INC
O VOORHEES PAINTING CO
ACREE ELECTRIC SERVICE
EASTGATE PLUMBING INC
ME THOMPSON INC
MELTON ELECTRIC INC
CITY OF TRUTH MINISTRIES
CITY OF EAST BREWTON
THE AQUINAS MONTESSORI SCHOOL
AIRPORT RANCHES INC
JACKLYNS SALON SPECIALISTS
BACLIFF VOLUTEER FIRE DEPARTMENT
ACCESSORIES BY SHAHARA
PREMIERE ENTERPRISES
SALEM COMMUNITY CHURCH OF GOD INC
SCHMALING GLASS COMPANY INC
SHIELDS SOUTHEAST SALES INC
IUP RESEARCH INSTITUTE
DESOTO ELECTRICAL PRODUCTS INC
MR TURF INCORPORATED
MR TURF INC
WEST VALLEY WELDING INC
FOUR GUYS AUTO INC
D  A BUILDING SERVICES INC
GL USED CARS
HURLBUT ROBERT N JR CARPET INSTALLATION
R N H FLOORING
ROBINSON WASTE SERVICES INC
PERRYSBURG MOVING  HAULING
PAW RESTORATION LLC
SOUTH TEXAS GRAPHIC SPECIALITIES INC
DUANE MOYER MECHANICAL SERVICE
GARDNER ENTERPRISE GROUP LLC
ADVANCED MEDICAL THERAPY LLC
TRIWEST LTD
C  H PROP

WIRE TECH EDM INC
CELEBRITY AUTO REPAIR
WITTYS MUFFLER  ALIGNMENT CENTER INC
BIG COUNTRY CUSTOM HOMES INC
JJ DESLOGES INC
TRADE FINISHING INC
FURNITURE DISCOUNTERS INC
TRAITEMENT DEAU DE LESTRIE INC
HAWK EXPRESS INC
FREY WELL DRILLING INC
ROBERT WALL ENTERPRISES
LUCIDO ENGINEERING  SURVEYING LLC
J E SYSTEMS INC
JE SYSTEMS INC
NATIONAL POLISHING SYSTEMS INC
CHABAD OF YONKERS
LIFTSERVICE INC
ALLSTAR SOUND LLC
VTECH MANUFACTURING CO INC
ZYNGUS ENTERPRISES INC
NORTHLAKE PLUMBING SERVICE INC
HEATING  COOLING PROS INC
GLOBALIZE SERVICES LLC
L MCNEIL TRUCKING LLC
SOCORRO CONSOLIDATED SCHOOLS
NV2A GROUP LLC
CATO CONSULTING SERVICES LLC
ON POINT TRANSPORTATION
NORTHERN BAY CONTRACTORS  INC
APELIAN RUG CO INC
AMERICAN RENEWABLE ENERGY
JOSEPH SCHOOL DISTRICT 6
KELCO INSULATION INC
M  M ELEVATOR COMPANY
METRO ELECTRIC CED
INFORMATION SYSTEMS SOLUTIONS LLC
MT TECHNOLOGIES
SMITHS WATER AND SEWER AUTHORITY
A CARPENTERS SON
BHI ROOFING
GOLD CROWN CREDIT LLC
MARIEN INSURANCE INC
CANDELA CORPORATION
FUE

...

BLOSSOM ENTERPRISE LLC
ADELPHIA PLUMBING  HEATING CORP
ADELPHIA PLUMBING  HEATING CO
TOP THAT LLC
BELLWETHER ELECTRIC COMPANY
AMERICAN PRECISION MACHINING INC
WILSON MANAGEMENT  PROPERTY LLC
CONNOR CONSTRUCTION LLC
BROCK DOORS  WINDOWS INC
PLUMB PROPERTIES
ON THE MOVE REAL ESTATE INVESTMENTS LLC
MAP CROSS CONNECTIONS INC
OVER  UNDER LLC
HYTEC EQUIPMENT COMPANY LLC
HACKETTSTOWN MOTOR IMPORTS
PRIDE MECHANICAL LLC
TACTICAL OFFICE SOLUTIONS INC
MATAMOROS CONSTRUCTION CORPORATION
DOGWATCH OF METRO ATLANTA INC
THE SHORELINE LANDSCAPE COMPANY LLC
SMART ELECTRICAL SOLUTIONS LLC
ROLAND PLUMBING INDUSTRIES INC
ROLAND MECHANICAL INC A CORP
HOOKS INDEPENDENT SCHOOL DISTRICT
GRIMES LEGAL INC
DAN PAYNE
THE MATTUS COMPANY INC
FORBES SANITATION  EXCAVATION
FORBES SANITATION  EXCAVATION INC
GREEN MOUNTAIN MESSENGER
GREEN MOUNTAIN MESSENGER INC
VITAL DELIVERY SOLUTIONS LLC
COMBINE CONTRACTORS LLC
NEFF TOWING SERVICE INC
DYNAGRID CONSTRUCTION GROUP LLC
DYNAGRID CONSTRUCTION GROUP LL
MAISON MARC LECLERC L

CHOICE EQUIPMENT RENTALS LLC
PROPERTY2HOME INC
SIMPLEX SOLUTIONS LLC
ICEBERG AIR CONDITIONING  HEATING LLC
BANC TRUCKING LLC
KNESZ CERAMIC TILE INC
OILFIELD INDUSTRIAL SUPPLY OF LOUISIANA INC
DOORMAN COMMERCIAL LLC
RENT A HOME OF THE TRI AD INC
COUNTRYSIDE BAPTIST CHURCH OF GAINESVILLE INC
LARAMIE COUNTY SCHOOL DISTRICT 1
EMERALD LEGACY LLC
EAST GWILLIMBURY TOWN OF
LEGACY SPRINKLER SERVICE INC
HARRY E REED INSURANCE AGENCY INC
SOUTH ELECTRICAL INC
THE TROPHY HOUSE INC
THE TROPHY HOUSE
ADVANCED CABINETRY INC
DALROCK PLUMBING COMPANY
DOWNTOWN COMMUNITY SAFETY PARTNERSHIP INC
TEXTON INC
INDUSTRIAS CH SAB DE CV
REDLINE PIPELINE LLC
BLUE SKY LAWN CARE LLC
MARQUEZ MEDICAL SUPPLY LLC
GLOBAL BUILDERS LLC
CULP CONSTRUCTION COMPANY
JARISMA SERVICES LLC
CERTIFIED COCO LLC
LARKEY DISTRIBUTING INC
GREAT FALLS PRERELEASE SERVICES INC
LAMBS LANDSCAPING
COMTEC INCORPORATED
MEDEK LLC
MAGNESIUM ALLOY PRODUCTS COMPANY INC
H S A INC
DE LA SALLE HIGH SCHOOL
FEDUKE MOTOR COMPANY LLC
GRAHAM LAWN CARE
S2F SER

CENTER FOR CREATIVE ARTS INC
PALOMO S CONSTRUCTION
ARPESH INC
RETINA CONSULTANTS OF SOUTHWEST FLORIDA INC
WYCOFF DEVELOPMENT  CONSTR
BREADFARM INC
SPEARMAN PROPERTY GROUP LLC
PRISTINE LLC
USINAGE INDUSTRIEL RLS
TITAN PROPERTIES LLC
BAUER COMPRESSORS INC
E 3 ELECTRIC LTD
E3 ELECTRIC LTD
PANORAMA CUSTOM HOME BUILDERS INC
C H WINANS COMPANY
ELECTRONIC SERVICE SOLUTIONS INC
ELECTRONIC SERVICE SOLU 0486
EL DORADO ROOFING  CONSTRUCTION
LOCAL WASTE RECYCLING
MEAT FREE INC
SOLUTION BASED TREATMENT AND DETOX
MRS G DIVING SPECIALISTS INC
IRIS MOBILE AUTO REPAIR LLC
ACS ENTERPRISES LLC
KROWN 1 REGISTRY
CARDINAL MCCLOSKEY SCHOOL  HOME FOR CHILDREN
TINT WORLD ENTERPRISES LLC
CLARATY LLC
BYJAZMINENICOLE LLC
LINDSEY MANUFACTURING CO
ASSET SPECIALISTS INC
HIRE POLICE
RYAN PLUMBING AND HEATING OF ST PETER INC
SOUTHWEST MECH
CENTRAL MATERIALS LLC
ASTRO CONSTRUCTION DRYWALL LP
BELCO CABINETS INC
DGC GLASS SYSTEMS LLC
FACILITY ENGINEERING SERVICES CORPORATION
J  P PRECISION DEBURRING INC
ANHING CORPORATIO

A  A ELECTRIC
CRONSHAW CARPENTRY INC
PIT STOP BEVERAGE
THE EXCHANGE BANK  TRUST INC
DG ELECTRIC LLC
UNITED PROPERTIES CORP
KINGS III OF AMERICA LLC
BLUE DIAMOND PAVING INC
AREA AGENCY ON AGING OF SOUTHEAST ARKANSAS INC
PRO EARTHWISE
PERRYTON HERALD
BREITER H WINDOW CLEANING LTD
PRIDHAMS STUDIO LIMITED THE
SOUTHERN LANDSCAPE PROFESSIONALS INC
LANDMARK OF DESOTO
DIVERSIFIED ENGINEERING  MANUFACTURING INC
KIMBERLY G LASHES INC
B  G PROPERTY SERVICES INC
PERRYSBURG SCHOOLS TECHNOLOGY
VERITAS PEST MANAGEMENT LLC
VENTURE SYSTEM GROUP INC
DA SHOP LLC
BRAD BROWN ELECTRIC
MOTIVATIONAL TRANSPORT LLC
KINGS RIVER CONSERVATION DISTRICT
ACCLAIM CHARTER SCHOOL
LOTS O BEER LLC
A  G HEALTHCARE STAFFING AGENCY LLC
ADVANCED ELEMENTS INC
STAH CORP
OCEANSIDE GLASSTILE COMPANY
TOTAL LASER CARE OF NC INC
ARNOLD CHEVROLET LLC
SINAANOTOA LLC
PLAYCARE INC
LONG NINE INC
TWT LEASING INC
RONCO FABRICATORS LLC
ANDERSON UNIVERSITY
ALL POWER CONSTRUCTION CORP
ANUEVIZION LLC
GEOTHERMAL DRILLING INC
VINTAGE WINE COMPAN

KeyboardInterrupt: 

In [19]:
for w in list_.values:
    print(w.word, w.count)

BERGE 1
TRANSPORT 161
COUNTY 210
OF 892
MOBILE 57
IEC 1
INC 8152
BUYBACKBOY 1
LLC 5391
PJF 2
ENTERPRISES 399
ADVERTISING 10
BAUMGARDNER 1
MOTORS 51
MULTITECH 1
SEALERS 1
SINAK 1
PLUMBING 346
CO 769
TRICOUNTY 9
ENGINEERING 106
THE 444
ELLIOTT 4
COMPANY 632
SERIOUS 1
GRIP 2
AND 742
ELECTRIC 408
LT 2
FOOD 36
DISTRIBUTOR 3
WALL 13
FX 3
LTD 420
PERFORMANCE 17
TESTING 20
HERITAGE 15
HAVEN 16
INSANELY 1
FAST 6
HOME 257
SOLUTIONS 324
SANITATION 7
TK 2
VISION 10
HOLDINGS 99
INCORPORATED 157
PAYNE 4
REFRIGERATION 47
ELECTRICAL 153
MICRO 4
INFORMATICA 1
MARATHON 3
FLORIST 13
SPINNING 1
CRANE 20
WORKS 59
FALLS 13
METAL 66
FABRICATORS 11
INDUSTRIAL 82
SERVICES 998
MCKAY 2
ASSOCIATES 179
UNTOUCHABLES 1
TRANSPORTATION 181
PWC 1
MANAGEMENT 243
KOLDAIR 2
HEATING 302
AIR 272
CONDITIONING 111
CONDITIO 2
MECHANICAL 203
CONTRACTORS 123
SOLID 9
GROUND 9
LANDSCAPE 122
CONSTRUCTION 778
RUNWENATCHEE 1
EXPLORE 1
JLM 1
IL 2
VAN 15
SARK 1
READING 2
CORNER 1
REDMOND 2
REACH 2
BLANKENSHIP 1
ENDURANCE 2
MACHINE 62
H

EASTER 3
SEAL 9
MAHONING 2
TRUMBULL 2
COLUMBIANA 2
COUNTIES 3
SCIENCES 2
FETTY 1
PSK 1
FUNSTAYZ 1
SITE 9
BARNEYS 1
PUMPS 6
CURLS 1
COMPAGNIE 1
CANADIENNE 1
DE 49
MTAL 1
FORG 1
PRUITT 2
ARLINGTON 4
ALFARO 1
HR 2
ALLEN 15
ENERPAC 1
BOBA 1
TRIPLE 16
XRAY 1
SHREVEPORTBOSSIER 1
LSP 2
WIND 10
GENERATION 7
RIGHT 9
MD3 1
KENNEY 2
PHINAZEE 1
RIDGEWAY 1
JALILIE 1
ELDRIDGE 2
SCI 1
GPJ 1
ARANSAS 1
WAYS 2
SCHRAGGER 2
LAVINE 1
NORTHERN 31
ESSCENTIALS 1
HELPING 5
HANDS 16
CE 5
GODDESSWAYINNER 1
GODDESS 2
ALLSTEELHOMECRAFT 1
FERGUSON 3
ALPINE 8
CABINET 12
EXPRESS 82
ENERGY 67
OPERATING 8
CLOSETS 4
MORE 24
YOUNGS 1
JMC 2
STONE 47
LIFTEC 1
INTERCONNECT 1
ENGLAND 17
ABBEVILLE 1
MILLING 4
KNOX 4
KERSHAW 2
LLOYD 2
SCHNEIDER 3
RANCH 22
WILSON 14
ROCHELLE 1
GROUNDSCAPE 1
BLACKLAND 1
PACKAGING 13
CG 4
MASONRY 38
ABLE 8
TAPE 2
KCJ 1
KENNEDALE 1
INDEPENDENT 30
GT 3
EDGAR 1
DREAMS 7
ABRASIVES 1
CHATMOSS 1
CABLEVISION 2
PROGRAMMABLE 1
DEVICES 1
JACKSON 17
LAND 41
CHESAPEAKE 9
LANDERS 2
MCLARTY 3
OLATHE 2
MAGDALEN

MARSILLIOS 1
VIDEO 10
DREAMBIGGERTRUCKING 1
KELLEMS 1
RD 6
MATTEX 2
POUND 2
BROOKHAVEN 1
MAUPIN 1
TURN 3
AM 9
AID 2
MAR 2
PANDIA 1
MAHER 1
CHAUFFEURS 1
OFFERING 1
PROTECTIVE 15
VAZ 1
FORREST 5
ILEGEND 1
MULTISERVICE 2
REDEFINING 1
EXPECTATIONS 1
BYRUM 1
CBFA 1
GLANDER 1
WILCOXSON 1
FAIRHAVEN 2
TRANSLIFT 2
DOCK 8
LUXE 2
PELLE 1
MINDLESS 1
CRAFTING 2
DAMIAN 1
BARKER 2
ROADS 2
WATLEY 1
GREENSWARD 1
MARVEL 1
BLACKOUT 2
SEALCOATING 5
BUDGET 4
ANGEL 6
OTTEN 1
KINGDOM 4
MORRIS 9
COLLISION 23
BODY 45
GRAF 1
TOWNS 1
REPROGRAPHICS 2
BUSY 4
SHAS 1
WELLNESS 8
IMPROVEMENT 13
HYFLODRAULIC 1
ARCEMENT 1
ALLN1 3
CONDO 3
GREENWOOD 2
EXPERTS 13
SWAROVSKI 1
BOXBORO 1
REGENCY 6
HOTEL 11
CONFERENCE 3
MERSE 1
SNELLS 1
LIMBS 1
BRACES 1
SHREVEPORT 2
SUND 1
METHODIST 6
PRECON 1
TENNESSEE 4
MONITORING 3
BAY 34
GENEST 1
LINEN 2
PARAMOUNT 11
PEARSONS 1
MISSOURI 5
EXCELSIOR 4
SENIOR 13
PRIVATE 9
LENDING 2
JAM 2
WASHBURN 2
GREENES 1
GILLS 1
LEXINGTON 3
GALLARDO 1
24HOUR 1
GILBERT 4
ROSES 1
BEAUTIFUL 3
DIVINE 10
RESO

WEBER 5
COMPASSIONATE 1
HEAL 1
INVISIBLE 1
VENTURA 5
EPIC 3
MULTILAKE 1
MULTI 4
MCCALL 2
DONNELLY 1
YC 1
COLLECTION 12
GALVESTON 1
MUD 2
STAT 3
SHED 3
PEOPLE 8
MONTEREY 4
KAM 1
ODHNER 2
ROTHROCK 1
MOUNDVILLE 2
MCLAIN 2
BABCOCK 1
FICEK 1
COMMUNICATION 12
WTIC 1
ALBION 3
SCHWARK 1
SOULISTIC 1
FROST 3
BARBER 5
JS 12
WEINMANN 1
MIKE 19
KELLY 11
WINGFIELD 2
CONSTRUCTIO 1
LEMCKE 1
TRENZ 1
DUTCHMAN 1
DRAINS 5
SOCONSULTING 1
HAITZ 1
ESTRADA 1
CARLYLE 2
MEI 1
GNY 1
ASIGMA 1
BIOSTYLE 1
HEROLD 1
MMM 1
ENJOY 1
ASTEK 1
WALLCOVERING 5
CROSSROAD 2
PRINT 7
DONOVAN 2
LANES 4
ROYALTY 3
FANCY 2
FEATHERS 2
GRANTBLACKFORD 1
MENTAL 11
BECO 1
LIMITE 1
BOULDEN 1
OUTLET 6
BOBS 5
LOADING 2
HAPPY 5
POND 1
BEARPOND 1
KOQ 1
DIANE 2
ENCOMPASS 3
RAPPAHANNOCK 1
MENS 4
HERSHAL 1
NAVE 1
BOLTONS 1
DAX 1
BERING 1
STRAITS 1
NATIVE 5
BLOCLEVEL1 1
GETAWAYS 1
MARLBOROUGH 1
WAYNES 2
REHABILITATIVE 1
ORAL 2
SUFFOLK 1
SURGERY 5
RAWLINS 1
AUGUSTINE 2
FR 1
MERIT 3
WOODWORKING 7
GEIST 1
SPOT 3
AN 2
ALMOND 1
BOULANGERIE 1
BLOUIN 1


INTERIORMOTIONS 1
SUNVISON 1
HOMESTEAD 2
GACAROLINA 1
DROP 1
SHOPCONSIGMENT 1
LINX 1
GAIDAS 1
SECUREONE 1
HEWITSON 1
BROOKMYER 1
BOOKKEEPERS 1
MAC 11
AMERI 1
GTA 2
AGUA 1
VERIFIED 1
SIGNINGS 1
DARLING 3
INGREDIENTS 2
REFRIGERANT 1
MAINEWAY 1
MARRA 1
ESP 1
HERE 1
YOU 8
ERRANDS 1
FOOTHILL 1
KAWAHARA 1
GOODWIN 4
GERMAIN 1
COMMUNITIES 6
GEORGIA 17
CHAMBERLAIN 2
GROC 1
EXPLORATION 1
PRODUCTION 14
ALABAMA 11
BATH 11
MELDRUM 2
BROADWAY 2
CORNELL 1
PETITS 2
PAINS 1
MECHANICALELECTRICALPLUMBING 1
BOILING 2
SPR 1
ENTEGRA 1
PP 1
ROPERTY 1
MANAGERS 2
CLEVELAND 3
BRICKS 1
AMINA 1
RENIER 1
AMA 1
TAMPA 10
DICKS 2
ITHACA 2
HIWAY 1
HIFI 1
FL 10
JOBIN 1
ORGANIZATION 5
PFAFF 1
SEBASTIAN 2
EXOTHERM 1
SPRAY 7
FOAM 8
LIAS 1
TOWNING 1
EDMONDS 1
PALATE 1
SERVICEMAX 1
BUTTE 3
DALESSANDRO 1
COATES 2
EXCELLENCY 1
MARSAN 1
CHARGED 1
HVACR 2
HYATTSVILLE 1
PH 1
ERNESTO 1
JOSE 3
CARDONA 1
WESTSIDE 4
RAILCAR 1
BLESSED 3
FELKER 1
GIGAFY 1
TULSANS 1
UNITY 6
CREATING 1
BRODY 2
RONISTAL 1
HOLMES 7
FURGESON 1
HELLER 1
OR 

...

REDEVELOPMENT 2
CLAY 2
NUVO 1
KUNTZ 1
SLACK 1
ENNIS 1
SQUIRE 1
CANE 2
OFF 4
PATAGONIA 1
COLFAX 1
NCAL 2
MORRELL 1
PENTUCKET 1
MOLINA 2
BIOMED 1
HOLTONARMS 1
NEXUSWOODBOURNE 1
OGRADY 1
SPRIGGS 1
MCCOYROCKFORD 1
BROXSONS 1
GHOST 1
FIFTEENTH 1
HIGHSPIRE 2
MEDIRYDE 1
PLASTER 2
AKCONSTRUCTION 1
VISIONZ 1
VM 1
ROBT 1
BAGGETT 1
CHIPPEWA 4
DEPARTM 1
WBG 1
ROOS 1
AIKENS 1
RUNNER 2
ASSEMBLIES 1
DOWNEY 2
IMPEX 1
ISOTOPES 1
OWOSSO 1
ZENITH 1
CAIN 1
EDOS 1
MANUFACTURERS 1
REPRESENTATIVES 1
GLENBURN 1
CHIEF 2
SYSTMES 1
WESPAT 1
BLACKWRAPMEDIA 1
COLBERT 1
MULINO 1
MORGANTON 1
BEAUCAL 1
RUST 1
AYALAS 1
TRUEWEST 2
EGREEN 1
DRAPERIES 2
LANDTECH 1
WMI 1
KIMBERLY 2
STANCIL 2
SEINER 1
NAK 1
CARDIFF 1
ACHIEVEMENT 1
SPIRITMAN 1
SHIRTS 2
BLESSEDHANDZZSEATZZNMORE1 1
BADCOCK 1
FURNISHINGS 2
PEACH 1
VAUGHN 1
ALLNINVESTMENTS 1
JUNIOR 2
AIMS 1
TEMPORARY 1
JTA 1
ALDERCENTER 1
KINETIX 1
IMPORTING 1
RAHL 1
JJA 1
BANGOR 2
ELIZABETH 3
HENCH 1
EDIFIED 1
BERKHEIMER 1
CUE 1
MANUEL 2
NAVARRO 1
SPENCER 1
JACE 1
WILLOWBROOK 

EMPOWER 2
WEAR 1
TROPHY 5
EMINENCE 1
HOLDINGZ 1
HEATINGINC 1
RASMUSSEN 1
JORY 1
PLANET 1
ELDERLY 2
BYROMDAVEY 1
BYROM 2
DAVEY 1
MOREAIR 1
WAYNEALLEN 1
IMPAKT 1
LYDIA 1
ALLCO 1
RALPHO 2
ELLIOTTS 1
AUCHTER 2
EVO 1
SCRUGGS 1
SNOORK 2
EXHAUSTCLEAN 1
ARMED 1
FORCES 1
ADCOGRIER 1
HERCULES 2
GUNTHERS 1
NURTEC 1
FRIENDLY 2
PUMPERNICKS 1
WYCOMBE 1
HORNUNGS 2
AUTHENTEKA 1
JEM 1
FRS 1
RESTAGE 1
ORAINE 1
ROUSE 1
MARSPEC 1
CUTLER 1
AMAC 1
AMES 2
BLANK 1
FABRICS 3
PURINTON 2
SMOKES 1
EATS 2
VO 1
MAIDPURE 1
CHAMPS 1
CDS 1
VINELAND 1
KABLAN 1
SASHEIS 1
ALSTON 1
PRETTYFACETRANSPORT 1
THINAIR 1
FOLIAGE 1
COSENZA 1
RCL 1
BORGES 2
TEMECULA 1
ACERRA 1
CLEBURNE 2
EWM 1
BIBLE 1
PEMER 1
VERMONT 1
KLETTE 1
TEA 2
KALWIL 1
CHILDS 2
ROSELINE 1
DOZIER 1
CISILSKI 1
BUGABOO 1
ZACCAGINNO 1
MOSESIAN 1
PROTETION 1
BALM 1
GRACE 6
FIVESTAR 1
ROCKING 1
LICITRA 1
HUBENY 1
BIOWASTE 1
ACCUNEERING 1
ZIMMERMANS 1
PURINA 1
NORTHFIELD 1
DUNKINS 1
JERL 1
WINNSBORO 1
GNOSKO 1
JRK 1
LABORDES 1
TELE 2
SOWERS 1
BUBBAS 1
MINNIE 1
TAST

INDEFINITE 1
STRIP 1
MALLOY 1
ABBY 1
BRISLIN 1
TRAILCO 1
SEWCAL 1
CRONIN 2
DILLON 1
BOTT 1
HYB90 1
SANDHU 1
IMPORTERS 1
TECHNIPC 1
INFORMATIQUE 1
YORKVILLE 1
CONG 1
BNAI 1
JEHUDA 1
CAUSE 1
HAQUE 1
ADC 1
RINGS 1
PTS 1
ACCENTS 2
SERVI 3
ENKI 2
CALUEDGFC 1
FORTUNE 1
MUNOZ 3
CALLOW 1
STU 1
SOUTHGATE 1
HOUSES 1
RDL 1
INSULATIONS 1
DETRICK 1
MITCHELLS 2
SANFORD 1
ABERS 1
HUTTONS 1
SCATAS 1
PUREWORLD 1
BRALEYS 1
EISEL 1
ZEUS 1
ROYALHALL 1
CORTE 1
MADERA 1
CEM 1
ROTAFLOW 1
CATHERINEELIZABETH 1
CAD 1
GEARSHIFT 1
HOLLISCO 1
BROOKHEART 1
POOR 1
BOATYARD 1
JAR 1
TIPPA 1
HOPLER 2
BURTONS 1
WILLIES 2
DUSTLESS 1
KIRA 1
SYNTHETICS 1
DOMENICO 1
RIMS 2
YUM 1
ENTERPRISESES 1
TECNICA 1
THERMALL 1
WEXFORD 1
PRATER 2
SHENEL 1
FCL 1
BLUESTREAK 1
LOCKSMITH 3
CORONET 1
ARP 1
HELIFLITE 1
HELI 1
FLITE 1
GEOENGINEERS 1
SVETNESS 1
DREKIA 1
RAS 1
RETAILER 1
BAYSTATE 1
OLSSON 1
EMMA 1
SELVINS 1
HELCO 1
POORTENGA 1
ARENA 2
TSP 1
CYRIACKS 1
ENVIROMENTAL 1
ONEILLZ 1
MAINTENAN 1
GREGORYS 1
LAMINATE 1
YESHIVATH 1
SHAAR 1

CARLOVSKY 1
DIMOND 1
CANOPY 1
CNC 1
DONS 2
ICART 1
DESIGNZ 1
BOHRENS 1
NUBIANS 1
LIFELINK 1
RELIBLU 1
RAGAN 1
DIG 1
ARMITAGE 1
FECT 1
SEELY 1
XMS 1
WOOTTEN 1
PRINCIPLE 1
GOODWYN 1
CTC 2
TIMESAVERS 1
ACQUISITON 1
LOUPE 1
BARRERA 1
ELLINGSWORTH 1
INFANT 1
FORMULA 1
GUILFORD 1
PARKERD 1
LANGS 1
SPORTING 2
RMG 1
YOGI 1
BEARS 1
JELLYSTONE 1
SULLY 2
EWING 2
GH 1
KWIK 1
KREDITS 1
CIPHER 1
COUNTERS 1
FOOT 1
KINGSWAY 1
MEDLOGIX 1
DAVONIA 1
CREATES 1
THEO 2
GORSKI 2
SKYY 1
TRAXX 1
LIGHTYEAR 1
FLEXX 1
TYLIE 1
GAUGE 1
BENSALEM 1
TRANSAXL 1
TRANSAXLES 1
LIA 1
NORTHAMPTON 1
CJDR 1
PATULSKI 1
SHAMOKIN 1
FILLER 1
SPELT 1
BILLYS 2
CRABS 1
EMERICK 1
COLLECTIVE 2
HOFF 1
MOBBEZ 1
CAMERA 1
GOTHIC 1
CINEREP 2
EAZ 1
SWALLEY 1
ULTRASOUND 3
ANACAPA 1
SINGH 1
DJJ 1
TRANSKO 1
OVERALL 1
S3 1
GRIDLEY 1
GRAVES 1
ERA 2
SARAH 1
MENDAK 1
RAINFLOW 1
YEARS 1
KLAPPRODT 1
PORTES 1
MANOIR 1
EASHUM 1
PDD 1
MYXER 1
SEETHRU 1
MACKENZIE 1
FABWORKS 1
CHIMCHEREE 1
DEIDA 1
BARONLONG 1
MADE 3
NONPROFIT 1
TALLEYRIGGINS 1
ZAMMIT 1
T

BETHANIA 1
TISSA 1
MONTMORENCI 1
COUCHTON 1
MONTMORENCICOUCHTON 1
AVMARIE 1
GWINNUP 1
PARAS 1
APY 1
REISS 1
MACWELL 1
PVO 1
DOWNHOLE 1
TARKOWSKY 1
PIPER 1
LPA 1
BURDOL 1
WOOLDRIDGE 1
SERBER 1
HINSHAW 1
CULBERTSON 1
THESE 1
TREADMILL 1
PROVENCE 1
ECUABEVERAGE 1
TRIMM 1
ACADEMIC 1
ASTRO 2
SCHULERHAAS 1
TUMWATER 1
ABF 1
DNS 1
ANTUZZI 1
SURFACING 1
HOLMESVILLE 1
STALEY 1
JAN 1
REVETEMENT 1
PEINTURE 1
DMB 1
RICULFY 1
VASSILAROS 1
BELLEW 2
HIPSTERGANG 1
SHENANGO 1
VAUGHANBASSETT 1
KEARSLEY 1
PIERCING 1
BELMAR 1
STECHO 1
FULTON 2
MARULLO 1
DELIVERANCE 1
SCHERR 1
WACONIA 1
CHICARE 1
FAHMY 1
LOGUE 1
CONRAD 1
BERNARDS 1
CAMOCO 1
CONNELLY 1
HUTTO 1
DEVELOMENT 1
AUTOWISE 1
HUSH 1
MIN 1
LIS 1
LOLO 1
WILMETTE 1
VONCHER 1
AVLIS 1
DEBURRING 2
MASTRIANNI 1
RENOVATORS 1
BUIL 2
SHAWANO 1
MARRONE 1
SARTEK 1
CLEMENTIS 1
TRUSTE 1
KRUNA 1
KENWALT 1
MDC 1
RAPSON 1
MIKO 1
EARLE 1
FIXXER 1
COVARRUBIAS 1
AYUDA 1
JWP 1
CONSU 1
HUMMEL 1
KAUFFMANS 1
VICKI 1
WOGTREE 1
MERCENARY 1
LPG 1
FRAZIER 1
MAGEE 1
AIG 1
GOUGHS

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

WORD_LIST_df = BY_ACCOUNT_df # For this sample code, simply copy input to output


# Write recipe outputs
WORD_LIST = dataiku.Dataset("WORD_LIST")
WORD_LIST.write_with_schema(WORD_LIST_df)